<a href="https://colab.research.google.com/github/Nakib-Nasrullah/Heart_disease/blob/main/ecg_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imoort libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import(
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, auc
)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/ecgdataset.csv')

In [14]:
# Basic exploration
print("Dataset shape:", df.shape)
print("n\First 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())


Dataset shape: (7610, 18)
n\First 5 rows:
   Record_no  patientId  patientAge   TestDate  Cycle  DayCount  P_wave  \
0       1002        100          22  6/24/2022    100         1      80   
1       1003        101          20  6/24/2022    100         1      78   
2       1004        102          22  6/24/2022    100         1      78   
3       1005        103          23  6/24/2022    100         1      78   
4       1006        104          24  6/24/2022    100         1      78   

   Q_wave  R_wave  S_wave  T_wave  PR_Interval  QRS_Complex  RR_Interval  \
0      58     146      51      73         0.22         0.07          0.5   
1      57     142      50      71         0.21         0.07          1.0   
2      57     142      50      71         0.21         0.07          0.9   
3      57     142      50      71         0.21         0.07          0.9   
4      57     142      50      71         0.21         0.07          0.9   

   QT_Interval  QTc_Interval       Disease  Risk_l

In [15]:
print("\nBaasic statistics:")
print(df.describe())

print("\nCheck for missing values:")
print(df.isnull().sum())


Baasic statistics:
         Record_no    patientId   patientAge   Cycle     DayCount  \
count  7610.000000  7610.000000  7610.000000  7610.0  7610.000000   
mean   4432.823390  3904.500000    22.556110   100.0     1.499080   
std    2084.020468  2196.962107     2.995299     0.0     0.500032   
min    1002.000000   100.000000    18.000000   100.0     1.000000   
25%    2449.250000  2002.250000    22.000000   100.0     1.000000   
50%    4350.500000  3904.500000    22.000000   100.0     1.000000   
75%    6252.750000  5806.750000    23.000000   100.0     2.000000   
max    8155.000000  7709.000000    47.000000   100.0     2.000000   

             P_wave        Q_wave        R_wave        S_wave        T_wave  \
count   7610.000000   7610.000000   7610.000000   7610.000000   7610.000000   
mean      66.677792     48.221551    121.654928     42.155716     60.654796   
std      505.526512    367.703410    919.135621    321.728603    459.574684   
min        1.000000    -43.000000      2.0

In [16]:
print("\nTarget variable distribution:")
print(df['Risk_level'].value_counts())


Target variable distribution:
Risk_level
0    6670
1     940
Name: count, dtype: int64


In [17]:
print("\nTarget variable distribution:")
df.columns = df.columns.str.strip()
print(df['Disease'].value_counts())


Target variable distribution:
Disease
Normal          6169
Tachycardia      502
Normal           501
Tachycardia      409
Bradycardia       29
Name: count, dtype: int64


In [19]:
# train (70%) validation (10%) test(20%)

#Separate input (X) and output (y)
X = df.drop('Risk_level', axis=1) # Everything except Risk_level
y = df['Risk_level']              # Only Risk_level

print("X has all patients information (17 colums)")
print("y has Risk_level (1 column)")

# Split: 80% for training, 20% for testing

#Train test split function is very importent for machine learning

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,     # 20% for test
    random_state = 20   # Same result every time # random_state mens in every time same data go to train and same data go to test.
)

print(f"\n We will train with: {len(X_train)} patients")
print(f"We will test with: {len(X_test)} patients")

X has all patients information (17 colums)
y has Risk_level (1 column)

 We will train with: 6088 patients
We will test with: 1522 patients
